In [1]:
!unzip /content/ner_dataset.csv.zip

Archive:  /content/ner_dataset.csv.zip
  inflating: ner_dataset.csv         


In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

--2022-11-20 08:16:16--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  44.5MB/s    in 97s     

2022-11-20 08:17:53 (44.5 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [3]:
!gzip -d cc.en.300.bin.gz

# Entity extraction using Fasttext and LSTM

## Import everything important

In [4]:
!pip install fasttext
import fasttext.util
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.3 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3161463 sha256=b810dc08c4ba0dd22ca583be808682ec40b61b9d05e59d182eed35f19ae0fd59
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [5]:
import joblib
import torch
import torch.nn as nn
# import transformers
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import model_selection

from gensim.models import FastText as ft
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from nltk.tokenize import word_tokenize

from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Some config

In [6]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 128
VALID_BATCH_SIZE = 128
EPOCHS = 5
EMBED_DIM = 300

MODEL_PATH = "./state_dict.pt"
TRAINING_FILE = "/content/ner_dataset.csv" # путь до файла ner_dataset.csv

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# fasttext_model = ft.load_fasttext_format("cc.en.300.bin")
fasttext_model = fasttext.load_model("cc.en.300.bin")

## Dataset

In [9]:
class EntityDataset:
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        tags = np.array(self.tags[item])
        
        # print("text", text)
        # print("tags", tags)
        
        ids = np.array([fasttext_model.get_word_vector(s) for s in text])
        ids_pad = np.vstack([ids, np.zeros((MAX_LEN - len(tags), 300))])
        
        # ids_pad, tags_pad, mask = np.zeros(MAX_LEN), np.zeros(MAX_LEN), np.zeros(MAX_LEN) 
        
        # ids_pad = np.zeros((MAX_LEN,))
        # i = 0
        # while ids_pad[i] != len(ids):
        #     ids_pad[i] = 
        # print(ids_pad.shape)
        tags_pad = np.zeros((MAX_LEN,), dtype=np.int)
        tags_pad[:len(tags)] = tags[:len(tags)]

        mask = np.zeros((MAX_LEN,), dtype=np.int)
        mask[:len(ids)] = 1

# реализуем паддинг: если текст меньше MAX_LEN, 
# забиваем матрицу нулями так, чтобы матрицы были одинакового размера для каждого текста   
# возвращаем матрицу с нулями, список тегов с нулями, и маску -- разметку, какие элементы 
# являются словами, а какие -- пустые поля (нули). Маска нужна, чтобы правильно считать лосс
# то есть, не учитывать в нем "пустые" части с нулями
        # print("ids_pad", ids_pad)
        # print("tags_pad", tags_pad)
        # print("mask", mask)
        return (torch.tensor(ids_pad, dtype=torch.float32),
                torch.tensor(tags_pad, dtype=torch.long),
                torch.tensor(mask, dtype=torch.long))

## Training and evaluation functions

In [10]:
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss

In [11]:
def acc_stat(pred, target, mask):
    mask = mask.bool()
    pred = torch.masked_select(pred, mask)
    target = torch.masked_select(target, mask)

    correct = 0
    total = 0
    
    for i in range(0, len(pred)):
        if pred[i] == target[i]:
            correct += 1

    for i in range(0, len(mask)):
        if mask[i] == 1:
            total += 1

    # correct = # сколько элементов угадано корректно
    # total = # сколько элементов было всего, не считая "пустых" с нулями
    return correct, total

пример того как должно работать

In [ ]:
acc_stat(torch.tensor([1,2,3,4,0,0,0,0]), torch.tensor([1,2,3,4,5,5,5,5]), torch.tensor([1,1,1,1,0,0,0,0]))

(4, 4)

In [ ]:
acc_stat(torch.tensor([1,2,3,4,0,0,0,0]), torch.tensor([1,2,3,4,5,5,5,5]), torch.tensor([0,0,0,0,1,1,1,1]))

(0, 4)

## Loss function and model

In [12]:
class EntityModel(nn.Module):
    def __init__(self, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5, bidirectional=False):
        super().__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob,
                            batch_first=True, bidirectional=bidirectional)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(
        self, 
        embeds,
        hidden
    ):
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
# здесь пропустим через дропаут и линейный слой
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        num_directions = 2 if self.lstm.bidirectional else 1
        print("num_directions", num_directions)
        h_zeros = torch.zeros(self.n_layers * num_directions,
                              batch_size, self.hidden_dim,
                              dtype=torch.float32, device=device)
        c_zeros = torch.zeros(self.n_layers * num_directions,
                              batch_size, self.hidden_dim,
                              dtype=torch.float32, device=device)

        return (h_zeros, c_zeros)

## Data processing

In [13]:
def process_data(data_path):
    df = pd.read_csv(data_path, encoding="latin-1")
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

    enc_tag = preprocessing.LabelEncoder()

    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag, enc_tag

## Training

In [14]:
sentences, tag, enc_tag = process_data(TRAINING_FILE)
# lens = []
# for i in sentences:
#     lens.append(len(i))
# print(pd.unique(lens))
# print(tag)
# print(enc_tag)

In [15]:
meta_data = {
    "enc_tag": enc_tag
}

joblib.dump(meta_data, "meta.bin")

num_tag = len(list(enc_tag.classes_))

(
    train_sentences,
    test_sentences,
    train_tag,
    test_tag

) = model_selection.train_test_split(
    sentences,
    tag,
    random_state=42,
    test_size=0.2
)    # делим на трейн и тест с помощью train_test_split

In [19]:
train_dataset = EntityDataset(
    texts=train_sentences, tags=train_tag
)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=1,
    shuffle=True, drop_last=True
)

valid_dataset = EntityDataset(
    texts=test_sentences, tags=test_tag
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1,
    shuffle=False, drop_last=True
)


In [16]:
def eval_model(model, valid_data_loader):
    h = model.init_hidden(VALID_BATCH_SIZE)
    losses = []
    
    correct_sum, total_sum = 0, 0
    
    for inputs, labels, mask in valid_data_loader:
        h = tuple([each.data for each in h])
        # отправим inputs, labels и mask на GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        mask = mask.to(device)

        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_fn(output, labels.flatten(), mask, num_tag)
        losses.append(loss.item())
        
        correct, total = acc_stat(torch.argmax(output, dim=-1).flatten(), labels.flatten(), mask.flatten())
        correct_sum += correct
        total_sum += total
    return losses, correct_sum / total_sum

In [17]:
hidden_dim = 512
n_layers = 2

model = EntityModel(num_tag, EMBED_DIM, hidden_dim, n_layers, drop_prob=0.5, bidirectional=False)
model.to(device)

lr=0.005
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [20]:
counter = 0
print_every = 10
clip = 5
valid_loss_min = np.Inf
writer = SummaryWriter('logs')


model.train()
for i in range(EPOCHS):
    h = model.init_hidden(TRAIN_BATCH_SIZE)
    
    correct_sum, total_sum = 0, 0
    
    for inputs, labels, mask in train_data_loader:
        counter += 1
        h = tuple([e.data for e in h])

        inputs = inputs.to(device)
        labels = labels.to(device)
        mask = mask.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_fn(output, labels.flatten(), mask, num_tag)     # вызываем функцию для подсчета лосса
        loss.backward() # и делаем обратное распространение ошибки
        correct, total = acc_stat(torch.argmax(output, dim=-1).flatten(), labels.flatten(), mask.flatten())   # вызываем функцию acc_stat
        correct_sum += correct
        total_sum += total

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()  # градиентный спуск
        
        if counter % print_every == 0:
            model.eval()
            val_losses, val_acc = eval_model(model, valid_data_loader)
            model.train()
            
            val_loss = np.mean(val_losses)
            writer.add_scalar('train/loss', loss.item(), counter)
            writer.add_scalar('val/loss', val_loss, counter)
            writer.add_scalar('train/acc', correct_sum / total_sum, counter)
            writer.add_scalar('val/acc', val_acc, counter)

            print("Epoch: {}/{}...".format(i+1, EPOCHS),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(val_loss),
                  "Train Acc: {:.6f}".format(correct_sum / total_sum),
                  "Val Acc: {:.6f}".format(val_acc))
                
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), MODEL_PATH)
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

num_directions 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

num_directions 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

Epoch: 1/5... Step: 10... Loss: 0.775183... Val Loss: 0.766635 Train Acc: 0.759827 Val Acc: 0.846498
Validation loss decreased (inf --> 0.766635).  Saving model ...
num_directions 1


...

Epoch: 5/5... Step: 1470... Loss: 0.147445... Val Loss: 0.121538 Train Acc: 0.959400 Val Acc: 0.961757
num_directions 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

Epoch: 5/5... Step: 1480... Loss: 0.125086... Val Loss: 0.121465 Train Acc: 0.959382 Val Acc: 0.961598
num_directions 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

Epoch: 5/5... Step: 1490... Loss: 0.149230... Val Loss: 0.121475 Train Acc: 0.959366 Val Acc: 0.961757


## Inference

In [21]:
meta_data = joblib.load("meta.bin")
enc_tag = meta_data["enc_tag"]

num_tag = len(list(enc_tag.classes_))

text = """
Alex is traveling to New York
"""

device = torch.device("cuda")
model.to(device)


with torch.no_grad():
# так как это инференс, выключаем расчет градиентов:    with torch.nograd
    inputs = torch.tensor([fasttext_model.get_word_vector(s) for s in word_tokenize(text)], dtype=torch.float32)
    inputs = inputs.unsqueeze(0).to(device)
    h = model.init_hidden(1)
    tag, h = model(inputs, h)

    print(
        enc_tag.inverse_transform(
            tag.argmax(-1).cpu().numpy().reshape(-1)
        )
    )

num_directions 1
['B-per' 'O' 'O' 'O' 'B-geo' 'I-geo']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  app.launch_new_instance()
